In [15]:
from google.colab import files
files.upload()

Saving archive (2).zip to archive (2) (1).zip


In [16]:
!unzip 'archive (2).zip' -d dataset

Archive:  archive (2).zip
replace dataset/test/cats/cat_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [17]:
!ls dataset/train
!ls dataset/test

cats  dogs
cats  dogs


In [18]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [19]:
train_path = 'dataset/train'
test_path = 'dataset/test'

In [20]:
from posix import listdir
def loadimagesnlabels(path):
  images = []
  labels = []

  for label in ['cats','dogs']:
    label_path = os.path.join(path,label)
    for img_name in os.listdir(label_path):
      img = cv2.imread(os.path.join(label_path,img_name))
      img = cv2.resize(img,(128,128))
      images.append(img)
      labels.append(label)

  images = np.array(images)
  labels = np.array(labels)
  return images,labels

In [21]:
X_train,Y_train = loadimagesnlabels(train_path)
X_test,Y_test = loadimagesnlabels(test_path)

In [22]:
X_train = np.array([cv2.cvtColor(img,cv2.COLOR_BGR2RGB) if len(img.shape) == 2 else img for img in X_train])
X_test = np.array([cv2.cvtColor(img,cv2.COLOR_BGR2RGB) if len(img.shape) == 2 else img for img in X_test])

In [23]:
X_train = X_train.reshape(X_train.shape[0],-1)
X_test = X_test.reshape(X_test.shape[0],-1)

In [24]:
label_encoder = LabelEncoder()
Y_train =  label_encoder.fit_transform(Y_train)
Y_test =  label_encoder.fit_transform(Y_test)

In [25]:
svm_model = SVC(kernel='rbf')
svm_model.fit(X_train,Y_train)
Y_pred = svm_model.predict(X_test)

In [26]:
accuracy = accuracy_score(Y_test,Y_pred)
print(f"Accuracy:{accuracy*100:.2f}%")

Accuracy:60.00%


In [27]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.1,1,10],
    'gamma':['scale','auto'],
    'kernel':['linear','rbf']
}

grid_search = GridSearchCV(SVC(),param_grid,refit=True)
grid_search.fit(X_train,Y_train)
Y_pred = grid_search.best_estimator_.predict(X_test)

In [ ]:
accuracy = accuracy_score(Y_test,Y_pred)
print(f"Accuracy:{accuracy*100:.2f}%")

Accuracy:62.86%


In [37]:
def img(imgpath):
  img = cv2.imread(imgpath)
  img = cv2.resize(img,(128,128))
  img = img /255
  img = img.reshape(1,-1)

  predictions = svm_model.predict(img)
  return 'Dog' if predictions[0] == 1 else 'Cat'

import matplotlib.pyplot as plt

print(img('/content/download.jpeg'))

Cat
